In [1]:
import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pathlib import Path

c:\Users\JAHNAVI REDDY\Desktop\RAG\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
### Read all the pdf's inside the directory
def process_all_pdfs(pdf_directory):
    """Process all PDF files in a directory"""
    all_documents = []
    pdf_dir = Path(pdf_directory)
    
    # Find all PDF files recursively
    pdf_files = list(pdf_dir.glob("**/*.pdf"))
    
    print(f"Found {len(pdf_files)} PDF files to process")
    
    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()
            
            # Add source information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'
            
            all_documents.extend(documents)
            print(f"  ✓ Loaded {len(documents)} pages")
            
        except Exception as e:
            print(f"  ✗ Error: {e}")
    
    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents

# Process all PDFs in the data directory
all_pdf_documents = process_all_pdfs("./data/pdf_files")

#all_pdf_documents

Found 3 PDF files to process

Processing: Cover_Page.pdf
  ✓ Loaded 1 pages

Processing: CSE R22 Syllabus.pdf
  ✓ Loaded 202 pages

Processing: jahnavi-reddy-f45874fc-d2b6-4d39-9ea1-2f5726a44208-certificate.pdf
  ✓ Loaded 1 pages

Total documents loaded: 204


In [3]:
### Text splitting get into chunks

def split_documents(documents,chunk_size=1000,chunk_overlap=200):
    """Split documents into smaller chunks for better RAG performance"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")
    
    # Show example of a chunk
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")
    
    return split_docs

chunks=split_documents(all_pdf_documents)
#chunks

Split 204 documents into 590 chunks

Example chunk:
Content: CVR COLLEGE OF ENGINEERING
Name: G JAHNAVI REDDY
Roll Number: 23B81A05CR
Section: CSE G
Year & Semester: BTECH 3rd Year 1st Sem
Subject Name: NUTRITION AND DIETRICS
Advanced/Additional Open Elective –...
Metadata: {'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': 'ReportLab PDF Library - www.reportlab.com', 'creationdate': '2025-08-17T12:42:29+00:00', 'author': 'anonymous', 'keywords': '', 'moddate': '2025-08-17T12:42:29+00:00', 'subject': 'unspecified', 'title': 'untitled', 'trapped': '/False', 'source': 'data\\pdf_files\\Cover_Page.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1', 'source_file': 'Cover_Page.pdf', 'file_type': 'pdf'}


In [4]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""
    
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise

    def generate_embeddings(self, texts: list[str]) -> np.ndarray:
        
        """Generate embeddings for a list of texts
        Args:texts: List of text strings to embed
        Returns:numpy array of embeddings with shape (len(texts), embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")
        
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings


## initialize the embedding manager

embedding_manager=EmbeddingManager()
embedding_manager

Loading embedding model: all-MiniLM-L6-v2
Model loaded successfully. Embedding dimension: 384


In [6]:
import os
import uuid
import numpy as np
import chromadb
from typing import List, Any


In [7]:
import faiss
import numpy as np
import uuid
import json
import os

class FAISSVectorStore:
    """FAISS-based vector store for fast similarity search."""

    def __init__(self, index_path="faiss.index", metadata_path="faiss_metadata.json"):
        self.index_path = index_path
        self.metadata_path = metadata_path
        self.index = None
        self.metadata = []
        self.dimension = None

        self._load_or_initialize()

    def _load_or_initialize(self):
        """Load existing FAISS index or initialize a new one."""
        if os.path.exists(self.index_path):
            print("🔄 Loading existing FAISS index...")
            self.index = faiss.read_index(self.index_path)
            self.metadata = json.load(open(self.metadata_path, "r"))
            self.dimension = self.index.d
            print(f"FAISS index loaded. Total vectors: {len(self.metadata)}")
        else:
            print("🆕 Creating new FAISS index...")
            self.index = None
            self.metadata = []

    def add_documents(self, documents, embeddings):
        """Add document chunks + embeddings into FAISS."""
        embeddings = np.array(embeddings).astype("float32")

        # Initialize index if first time
        if self.index is None:
            self.dimension = embeddings.shape[1]
            self.index = faiss.IndexFlatL2(self.dimension)

        ids = []
        for i, (doc, emb) in enumerate(zip(documents, embeddings)):
            uid = uuid.uuid4().hex[:8]

            # Store metadata
            entry = {
                "id": uid,
                "content": doc.page_content,
                "metadata": doc.metadata
            }
            self.metadata.append(entry)
            ids.append(uid)

        # Add to FAISS index
        self.index.add(embeddings)

        # Save the index + metadata
        faiss.write_index(self.index, self.index_path)
        json.dump(self.metadata, open(self.metadata_path, "w"))

        print(f"✅ Added {len(documents)} documents to FAISS")
        print(f"📌 Total vectors in index: {len(self.metadata)}")

    def search(self, query_embedding, top_k=5):
        """Search for top_k similar vectors in FAISS."""
        query_embedding = np.array(query_embedding).astype("float32").reshape(1, -1)
        distances, indices = self.index.search(query_embedding, top_k)

        results = []
        for rank, (idx, dist) in enumerate(zip(indices[0], distances[0])):
            if idx == -1:
                continue

            entry = self.metadata[idx]
            score = 1 / (1 + dist)

            results.append({
                "content": entry["content"],
                "metadata": entry["metadata"],
                "similarity_score": float(score),
                "distance": float(dist),
                "rank": rank + 1
            })
        return results


In [8]:
class RAGRetriever:
    def __init__(self, vector_store: FAISSVectorStore, embedding_manager: EmbeddingManager):
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query, top_k=5, score_threshold=0.1):
        print(f"🔍 Query: {query}")

        # Generate embedding for the query
        query_emb = self.embedding_manager.generate_embeddings([query])[0]

        # Search FAISS
        results = self.vector_store.search(query_emb, top_k=top_k)

        # Filter by score
        filtered = [r for r in results if r["similarity_score"] >= score_threshold]

        print(f"📌 Retrieved {len(filtered)} documents")

        return filtered


In [9]:
import os
from dotenv import load_dotenv
from groq import Groq

# Load environment variables
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")

# ✅ Initialize Groq client (no langchain_groq needed)
client = Groq(api_key=groq_api_key)
MODEL_NAME = "llama-3.1-8b-instant"   # active model (as of Nov 2025)

# --- Simple ChatGroq-like wrapper (drop-in replacement)
class ChatGroq:
    def __init__(self, groq_api_key, model_name, temperature=0.1, max_tokens=1024):
        self.client = Groq(api_key=groq_api_key)
        self.model_name = model_name
        self.temperature = temperature
        self.max_tokens = max_tokens

    def invoke(self, messages):
        # messages is a list of dicts or objects with .content
        user_text = ""
        if isinstance(messages, list):
            for m in messages:
                user_text += getattr(m, "content", str(m)) + "\n"
        else:
            user_text = str(messages)

        response = self.client.chat.completions.create(
            model=self.model_name,
            messages=[{"role": "user", "content": user_text}],
            temperature=self.temperature,
            max_completion_tokens=self.max_tokens,
        )
        return type("Response", (), {"content": response.choices[0].message.content.strip()})

# ✅ use our wrapper instead of the broken langchain_groq
llm = ChatGroq(
    groq_api_key=groq_api_key,
    model_name=MODEL_NAME,
    temperature=0.1,
    max_tokens=1024
)

def rag_model(query, retriever, llm, top_k=5, min_score=0.2, return_context=False):
    results = retriever.retrieve(query, top_k=top_k, score_threshold=min_score)
    if not results:
        return {'answer': ['No relevant context found.'], 'sources': [], 'confidence': 0.0, 'context': ''}

    context = "\n\n".join([doc['content'] for doc in results])
    sources = [{
        'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
        'page': doc['metadata'].get('page', 'unknown'),
        'score': doc['similarity_score'],
        'preview': doc['content'][:300] + '...'
    } for doc in results]
    confidence = max([doc['similarity_score'] for doc in results])

    prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {query}\n\nAnswer:"""
    response = llm.invoke([prompt])

    output = {
        'answer': [line.strip() for line in response.content.splitlines() if line.strip()],
        'sources': sources,
        'confidence': confidence
    }
    if return_context:
        output['context'] = context
    return output


In [10]:
# 1. Load PDFs
all_pdf_documents = process_all_pdfs("./data/pdf_files")

# 2. Split into chunks
chunks = split_documents(all_pdf_documents)

# 3. Generate embeddings
texts = [doc.page_content for doc in chunks]
embeddings = embedding_manager.generate_embeddings(texts)

# 4. Store in FAISS
faiss_store = FAISSVectorStore()
faiss_store.add_documents(chunks, embeddings)

# 5. Create retriever
rag_retriever = RAGRetriever(faiss_store, embedding_manager)

# 6. Run RAG model
answer = rag_model("What is Engineering Chemistry About?", rag_retriever, llm)
for line in answer["answer"]:
    print(line)


Found 3 PDF files to process

Processing: Cover_Page.pdf
  ✓ Loaded 1 pages

Processing: CSE R22 Syllabus.pdf
  ✓ Loaded 202 pages

Processing: jahnavi-reddy-f45874fc-d2b6-4d39-9ea1-2f5726a44208-certificate.pdf
  ✓ Loaded 1 pages

Total documents loaded: 204
Split 204 documents into 590 chunks

Example chunk:
Content: CVR COLLEGE OF ENGINEERING
Name: G JAHNAVI REDDY
Roll Number: 23B81A05CR
Section: CSE G
Year & Semester: BTECH 3rd Year 1st Sem
Subject Name: NUTRITION AND DIETRICS
Advanced/Additional Open Elective –...
Metadata: {'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': 'ReportLab PDF Library - www.reportlab.com', 'creationdate': '2025-08-17T12:42:29+00:00', 'author': 'anonymous', 'keywords': '', 'moddate': '2025-08-17T12:42:29+00:00', 'subject': 'unspecified', 'title': 'untitled', 'trapped': '/False', 'source': 'data\\pdf_files\\Cover_Page.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1', 'source_file': 'Cover_Page.pdf', 'file_type': 'pdf'}
Generating emb

Batches: 100%|██████████| 19/19 [00:16<00:00,  1.13it/s]


Generated embeddings with shape: (590, 384)
🆕 Creating new FAISS index...
✅ Added 590 documents to FAISS
📌 Total vectors in index: 590
🔍 Query: What is Engineering Chemistry About?
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 46.35it/s]


Generated embeddings with shape: (1, 384)
📌 Retrieved 5 documents
Engineering Chemistry is a course that covers various aspects of chemistry and its applications in engineering. It includes topics such as:
- Water treatment and its importance
- Batteries and corrosion control
- Polymers and their engineering applications
- Petroleum and its products
- Engineering materials like lubricants, refractories, and smart materials
The course aims to provide students with a comprehensive understanding of chemistry and its practical applications in engineering, enabling them to become good engineers and entrepreneurs.
